# Chemical Kinetics and Numerical Integration

Here we will use methods of numerical integration to solve for the abundances of the H$_3^+$ isotopologues in the ion trap experiment from last week's notebook. After using integrated rate equations and curve fitting, we came up with this result:

![Deuteration Results](https://leeping.github.io/che155/assets/images/week05/deuteration_results.png)

The deviations, most notable in the D$_2$H$^+$ results, are because the reverse reactions were not included in our model. It would be very difficult to derive new rate equations, so we will use numerical methods instead.

## Forward Euler Method

First, we will reimplement the exact same model as last time, but this time we will solve using the Forward Euler Method. First, load in the `deuteration.csv` file. It contains the same experimental data as last week, but the time field has been rounded and lined up so that all abundances for each molecule are given at the same time values. This will make comparisons with the numerical models easier down the road.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

df = pd.read_csv('deuteration.csv')
    
df

As a reminder, the model was defined by the equations:

$$ \frac{\text{d}[\text{H}_3^+]}{\text{d}t} = -k_1[\text{H}_3^+] $$

$$ \frac{\text{d}[\text{H}_2\text{D}^+]}{\text{d}t} = k_1[\text{H}_3^+] - k_2[\text{H}_2\text{D}^+] $$

$$ \frac{\text{d}[\text{D}_2\text{H}^+]}{\text{d}t} = k_2[\text{H}_2\text{D}^+] - k_3[\text{D}_2\text{H}^+] $$

$$ \frac{\text{d}[\text{H}_3^+]}{\text{d}t} = k_3[\text{D}_2\text{H}^+] $$

We can express these in a simple form with the matrix equation:

$$ \begin{bmatrix} \text{d}[\text{H}_3^+]/\text{d}t \\ \text{d}[\text{H}_2\text{D}^+]/\text{d}t \\ \text{d}[\text{D}_2\text{H}^+]/\text{d}t \\ \text{d}[\text{D}_3^+]/\text{d}t \end{bmatrix} = \begin{bmatrix} -k_1 & 0 & 0 & 0 \\ k_1 & -k_2 & 0 & 0 \\ 0 & k_2 & -k_3 & 0 \\ 0 & 0 & k_3 & 0 \end{bmatrix} \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix} $$

Then, taking a time step $\Delta t$, we can compute new concentrations:

$$ \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i+1} = \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i} +  \begin{bmatrix} -k_1 & 0 & 0 & 0 \\ k_1 & -k_2 & 0 & 0 \\ 0 & k_2 & -k_3 & 0 \\ 0 & 0 & k_3 & 0 \end{bmatrix} \begin{bmatrix}[\text{H}_3^+] \\ [\text{H}_2\text{D}^+] \\ [\text{D}_2\text{H}^+] \\ [\text{D}_3^+] \end{bmatrix}_{\,i} \Delta t$$

As of Python 3.5, matrix multiplication (and other types of dot products) can be done with the `@` operator. When used with `numpy.ndarray` objects, the [`numpy.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) function is called. In our case, we will create a 4x4 matrix called `J` and a 1D array with 4 elements called `n` to store the abundances. When we call `J@n`, it multiplies each row of `J` by the 4 elements in `n`, and adds them up. Here we use the results from the curve fitting to ideally give us similar results as last time. We will set the step size `dt` to 0.1 ms, and take 1500 steps.

In [ ]:
#iniitialize rate constants
hd=6.3e10
k1=1.42e-9*hd
k2=1.33e-9*hd
k3=1.05e-9*hd

#H3+ at t=0 is 932, H2D+, D2H+, and D3+ start at 0.
n0 = np.array([932,0,0,0])

#initialize an empty 4x4 matrix, and plug in k values at the right places
J = np.zeros((4,4))
J[0,0] = -k1
J[1,1] = -k2
J[2,2] = -k3
J[1,0] = k1
J[2,1] = k2
J[3,2] = k3

#this array n will be updated with the new concentrations at each step. Initialize it at n0
n = n0
dt = 1e-4
steps = 1500

#this array will keep track of the values of n at each step
nt = np.zeros((steps+1,len(n0)))
nt[0] = n0

#take each steps, updating n at each one; store the results in the nt array
for i in range(0,steps):
    n = n + J@n*dt
    nt[i+1] = n
        
nt

Now we can plot the results and compare with the experimental data.

In [ ]:
fig,ax = plt.subplots()
t = np.linspace(0,150e-3,len(nt))

ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')

ax.set_xlabel("Time (s)")
ax.set_ylabel("Number")

lines = ax.plot(t,nt)
lines[0].set_color('#000000')
lines[1].set_color('#ffbf00')
lines[2].set_color('#022851')
lines[3].set_color('#c10230')
ax.set_yscale('log')
ax.legend()

Note that the step size is a critical parameter! If we increase the step size too much, we can get some bad results.

In [ ]:
n = n0
dt = 5e-3
steps = round(.15/dt)+1

nt = np.zeros((steps+1,len(n0)))
nt[0] = n0

for i in range(0,steps):
    n = n + J@n*dt
    nt[i+1] = n
    
fig,ax = plt.subplots()
t = np.linspace(0,len(nt)*dt,len(nt))

ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')

ax.set_xlabel("Time (s)")
ax.set_ylabel("Number")

lines = ax.plot(t,nt)
lines[0].set_color('#000000')
lines[1].set_color('#ffbf00')
lines[2].set_color('#022851')
lines[3].set_color('#c10230')
ax.set_yscale('log')

## Least Squares Fitting and Numerical Integration

It is possible (though not very common) to implement least squares fitting together with the numerical integration in order to estimate the kinetics parameters. We'll walk through the process here. Last time we used `scipy.optimize.least_squares`, which required us to calculate the residuals vector between the model and the experimental data. When using integrated rate equations, this was straightforward because we could just plug in the time for each data point into the model and compute the model's prediction. With numerical integration; however, we do not have such a function!

Instead, what we can do is save the model's outputs whenever the time matches the time at which an experimental data point is taken. If we choose time steps judiciously, we can make sure that we always sample the model at each time point needed. If we inspect the data frame, we can see that all of the time points are at a multiple of 0.1 ms.

In [ ]:
df

Therefore, a time step `dt` of 0.1 ms (or some integer factor smaller) will ensure that the model samples each time point we need to compare with the experimental data. The code below checks to see if `i` (the current time in units of `dt`) is in the array `tvals`, which is the time array converted to units of dt, and if so it stores the current model abundances in a list for later use. Importantly, this is chosen such that all of the time comparisons are between integers so that we don't have to worry about issues with floating point comparisons.

At the end of the clode block, `nm` is a 2D numpy array where each row is a time point and each column is the abundance of one of the ions.

In [ ]:
n = n0
dt = 1e-4
steps = 1500

nmodel = []
tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)

# print(tvals)

for i in range(0,steps+1):
    n = n + J@n*dt
    if i in tvals:
        nmodel.append(n)
        
nm = np.array(nmodel)
nm

In [ ]:
tvals

Now we'll plot the results. A quick side note here: we've been doing a lot of repetitive manual color changing. If you have a set of colors you want to consistently use, you can change matplotlib's default color cycling (see this [tutorial](https://matplotlib.org/tutorials/intermediate/color_cycle.html) for a quick example). Below I create a new `cycler` object that tells matplotlib to cycle between the 4 colors we have been using instead of its defaults. As the tutorial shows, you can either set the cycler on an `Axes` object like in the code below, which only affects that object, or you can apply the cycler to all subsequently created plots.

In [ ]:
from cycler import cycler

ucd_cycler = (cycler(color=['#000000','#ffbf00','#022851','#c10230','#266041','#8a532f']))

fig,ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
ax.plot(df['time'],nm,'o')

Now let's turn that into a function that takes the kinetics parameters (`h30`, `k1`, `k2`, `k3`) as arguments. We also need to pass the time values at which the model should be sampled, the step size, and the number of steps.

In [ ]:
def runmodel(params,tvals,dt,steps):
    h30 = params[0]
    k1 = params[1]
    k2 = params[2]
    k3 = params[3]
    n = np.asarray([h30,0,0,0])
    nmodel = []
    J = np.zeros((4,4))
    J[0,0] = -k1
    J[1,1] = -k2
    J[2,2] = -k3
    J[1,0] = k1
    J[2,1] = k2
    J[3,2] = k3

    for i in range(0,steps+1):
        n = n + J@n*dt
        if i in tvals:
            nmodel.append(n)
            
    return(np.array(nmodel))
    
    

Test to make sure the `runmodel` function works as intended:

In [ ]:
tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)
hd=6.3e10
k1=1.43e-9*hd
k2=1.33e-9*hd
k3=1.05e-9*hd
h30 = 932

runmodel(np.array([h30,k1,k2,k3]),tvals,1e-4,1500)

To perform the `least_squares` optimization, we need to create a function that computes the residuals of the model. This function must have the signature `f(x,*args,**kwargs)` where `x` is an array containing the parameters that will be optimized (`h30`, `k1`, `k2`, and `k3`), `*args` contains any additional arguments that are needed, and `**kwargs` can contain any other information.

Like last time, we'll use `**kwargs` to pass in the experimental data. `*args` will contain the `tvals`, `dt`, and `steps` parameters that need to be passed to `runmodel.` Ance we have the results of the model, we need to compute the residuals.

In [ ]:
def total_fit(x,*args,**kwargs):
    
    df = kwargs['df']

    nm = runmodel(x,*args)
    
    #a naive algorithm using for loops; slow, but flexible!
#    out = []
#    for i,model in enumerate(nm):
#        for j,mol in enumerate(['H3+','H2D+','D2H+','D3+']):
#            n = df.at[i,mol]
#            if np.isfinite(n):
#                out.append(n-model[j])
#    return out
                
    #taking advantage of numpy's array routines: fast, but requires more work if anything changes
    rh3 = df['H3+'] - nm[:,0]
    rh3 = rh3[~np.isnan(rh3)] #remove NaNs... isnan returns an array of booleans, so we take the logical not and use it as a slice to extract only the finite values
    
    rh2d = df['H2D+'] - nm[:,1]
    rh2d = rh2d[~np.isnan(rh2d)]
    
    #there are no NaNs in the experimental data for D2H+ or D3+
    rd2h = df['D2H+'] - nm[:,2]
    rd3 = df['D3+'] - nm[:,3]
    
    #concatenate and return
    return np.concatenate((rh3,rh2d,rd2h,rd3))
        

In [ ]:
total_fit(np.array([932, 1e-9, 1e-9, 1e-9]), tvals, dt, steps, df=df)

Now we can use `least_squares` to compute optimal parameters, and we can see that we get almost exactly the same results as the integrated rate equation approach. Note, however, that there is no problem with us starting out with `k1` and `k2` being equal! There is no divide by 0 error with numerical integration like there was with the integrated rate equations.

In [ ]:
import scipy.optimize as opt
import numpy.linalg as la

data = {
    'df' : df
}

tvals = df['time'].to_numpy()/dt
tvals = tvals.astype(int)
hd=6.3e10

result = opt.least_squares(total_fit,[900,1e-9*hd,1e-9*hd,1e-9*hd],
                           args=[tvals,1e-4,1500],kwargs=data,verbose=1)
pcov = la.inv(result.jac.T @ result.jac)

for i,x in enumerate(['[H3+]0','k1','k2','k3']):
    den = hd
    if i==0:
        den = 1.
    print(f'{x} = {result.x[i]/den:.2e} +/- {np.sqrt(pcov[i][i])/den:.2e}')

## Integration with `scipy.integrate`

Our manual implementation of the numerical integration uned the Forward Euler Method, whose total error is proportional to $(\Delta t)^{1}$. It is usually desirable to use a higher-order method to achieve either higher accuracy or obtain the same accuracy with fewer steps. The function we are going to explore is [`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html), which is made to solve initial value problems.

In [ ]:
import scipy.integrate as spi

spi.solve_ivp?

As we can see from the function description, we need to provide at least 3 arguments:
- `fun` is a function that computes the vector of derivatives. Its function signature needs to be `f(t,y,*args)`. `t` is the current time, `y` is the current state array (in our case, the array containing the molecule abundances), and the remainder of the arguments can contain anything else needed to compute the derivatives (e.g., rate coefficients, etc)
- `t_span` is a tuple that specifies the initial and final time for the integration
- `y0` is a vector containing the initial conditions - the starting abundances for the molecules.

In addition to those required parameters, there are three other optional arguments that are useful for us:
- `method` selects which numerical integration method will be employed. The default, `'RK45'`, is the fourth-order Runge-Kutta method, but several others are available, including some implicit solvers that are important when problems are "stiff." A system of equations is stiff when the solutions are very sensitive to the step size even when the solution appears "smooth." Chemical kinetics problems are frequently stiff when there are some very slow reactions combined with others that are very fast, and you want to evaluate the system over a long time compared with the rate of the fast reactions. In the current example, all of the reactions have comparable rates, so we will stick with `'RK45'`, but often the `'Adams'` or `'Radau'` methods are more appropriate for kinetics problems.
- `t_eval` is a list of times at which the model returns abundances. If this is None, the model only gives the results at the final time. If we pass an array of times, the results will contain the abundances at all of the time values specified in `t_eval` which fall within `t_span`
- `dense_output` causes the solver to construct functions that interpolate between time steps. This allows you to (approximately) evaluate the model at any time, not just at the time steps that were used in the model.

Note that nowhere do you need to specify the step size! All of the methods employ various algorithms to automatically determine the step size needed to bring the error down to a certain desired value. Some even include adaptive step sizes that can take smaller or larger steps depending on the magnitudes of the derivatives.

Let's re-implement the same model, but this time perform the integration with `solve_ivp`. First we need to write a function that computes the derivative.

In [ ]:
# function must take t and y as its first 2 arguments. Since our derivatives don't explicitly depend on t, that variable isn't used in the body of the function.
# to calculate the rates, we need the rate coefficients and abundances. The abundances are in y, so we need to pass the k values as arguments.
def calc_derivative(t,y,k1,k2,k3):
    
    J = np.zeros((len(y),len(y)))
    J[0,0] = -k1
    J[1,1] = -k2
    J[2,2] = -k3
    J[1,0] = k1
    J[2,1] = k2
    J[3,2] = k3
    
    return J@y

With that, we can now use `solve_ivp` to compute the solution from 0 to 0.15 seconds. We'll use the default `RK45` integrator, and set the `dense_output` flag to allow us to generate a quasi-continuous model function. In addition, we'll pass our `df['time']` array to `t_eval` so that we have the exact model values at the experimental time points.

Within the `result` object that is returned, we can access the dense solution with `result.sol`, which takes a time value as an argument. The solution values are in `result.y`, and the time points for each solution are in `result.t`. The plot that this cell creates shows both the dense output and the discrete solutions.

In [ ]:
hd=6.3e10
k1=1.0e-9*hd
k2=1.0e-9*hd
k3=1.0e-9*hd
h30 = 932

result = spi.solve_ivp(calc_derivative,(0,.15),y0=[h30,0,0,0],
                       t_eval=df['time'],method='RK45',
                       dense_output=True,args=(k1,k2,k3))
fig,ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
ax.scatter(df['time'],df['H3+'],color='#000000',label=r'H$_3^+$')
ax.scatter(df['time'],df['H2D+'],color='#ffbf00',label=r'H$_2$D$^+$')
ax.scatter(df['time'],df['D2H+'],color='#022851',label=r'D$_2$H$^+$')
ax.scatter(df['time'],df['D3+'],color='#c10230',label=r'D$_3^+$')
t = np.linspace(0,160e-3,1000)
ax.plot(t,result.sol(t).T)
#ax.plot(result.t,result.y.T,'o')
ax.set_yscale('log')

### Using object-oriented programming to make a generalized kinetic model

Up until now, our kinetic model consisted of hard-coded species and reactions.  When doing research, one might want to test different models quickly, which means a more general way of setting up kinetic models is desirable.  Below is a simple design of a reaction network class that stores the species in the system and individual reactions.

In [ ]:
class ReactionNetwork:
    
    def __init__(self):
        # Our constructor simply initializes some attributes to empty lists
        # so they can be extended by calling the other methods.
        self.species_list = []
        self.reaction_list = []
        
    def add_species(self, species):
        # Add a species to the list (can be "H3+" or "HD" for example)
        self.species_list.append(species)
        
    def add_reaction(self, reaction_str, kfwd=0.0, kbak=0.0):
        # Add a reaction to the list; the syntax is "H3+ + HD -> H2D+ + H2" 
        # The reactants and products are separated by " -> "
        # The species on either side are separated by "+"
        reaction_split = reaction_str.split("->")
        reactant_str = reaction_split[0]
        reactants = []
        for word in reactant_str.split(" + "):
            reactants.append(word.strip())
            
        product_str = reaction_split[1]
        products = []
        for word in product_str.split(" + "):
            products.append(word.strip())
        # The reaction list is stored as a list of dictionaries
        self.reaction_list.append({'string':reaction_str,
                                  'reactants':reactants,
                                  'products':products,
                                  'kfwd':kfwd,
                                  'kbak':kbak})
    
    def calc_derivative(self, t, concs):
        # Initialize the derivative to an array of zeros.
        derivatives = np.zeros_like(concs)
        
        # This dictionary maps the species name to the index
        # in the species list used to get the corresponding concentration.
        species_map = {}
        for i, species in enumerate(self.species_list):
            species_map[species] = i
        
        # Loop through each reaction.
        for reaction in self.reaction_list:
            # Calculate the forward rate as a product of the forward rate constant
            # and the reactant species concentrations.
            rate_fwd = reaction['kfwd']
            for reactant in reaction['reactants']:
                rconc = concs[species_map[reactant]]
                rate_fwd *= rconc
            # Do the same for the backward rate.
            rate_bak = reaction['kbak']
            for product in reaction['products']:
                pconc = concs[species_map[product]]
                rate_bak *= pconc
            # The net reaction rate.
            tot_rate = rate_fwd - rate_bak
            # Convert net reaction rate to rate of change of
            # individual concentrations.
            for reactant in reaction['reactants']:
                derivatives[species_map[reactant]] -= tot_rate
            for product in reaction['products']:
                derivatives[species_map[product]] += tot_rate
        return derivatives
           
    

With the class defined as above, we can now add our species and reactions, which can now go in both directions.  The `calc_derivative` method is defined with time as the first argument (even though the derivative has no explicit time dependence) because that's how `scipy.integrate.solve_ivp` needs the function to be defined.

In [ ]:
myNetwork = ReactionNetwork()
myNetwork.add_species("H3+")
myNetwork.add_species("H2D+")
myNetwork.add_species("D2H+")
myNetwork.add_species("D3+")
myNetwork.add_species("HD")
myNetwork.add_species("H2")
myNetwork.add_reaction("H3+ + HD -> H2D+ + H2", kfwd=1e-9, kbak=1e-10)
myNetwork.add_reaction("H2D+ + HD -> D2H+ + H2", kfwd=1e-9, kbak=1e-10)
myNetwork.add_reaction("D2H+ + HD -> D3+ + H2", kfwd=1e-9, kbak=1e-10)
myNetwork.calc_derivative(0, np.array([932, 0, 0, 0, 6.3e10, 0.0]))

We can now call `scipy.integrate.solve_ivp` to integrate the kinetic model using our method of calculating the time derivative. The other arguments to `solve_ivp()` are the time interval, the initial values of dependent variables (i.e. concentrations), and the time points to evaluate and store the dependent variables.

In [ ]:
import scipy.integrate
ts = np.arange(0, 0.15, 1e-4)
result = scipy.integrate.solve_ivp(myNetwork.calc_derivative, (0, 0.15), np.array([932, 0, 0, 0, 6.3e10, 0.0]), t_eval=ts)

The simulation results can now be plotted:

In [ ]:
fig, ax = plt.subplots()
ax.set_prop_cycle(ucd_cycler)
ax.plot(result.t, result.y[:4].T)
ax.set_yscale('log')
ax.set_ylim(0.01, 1000)

Using the new reaction network, you can now fit the model parameters to get better agreement with experiment.  Write a new function below that calculates the residuals using the new model.  (The cell below will be updated with the answer after the due date.)

In [ ]:
# Fill in this cell with your own code. You may reuse any of the codes from above.

The following codes will optimize the model parameters from a starting guess, assuming that your function is titled `total_fit_2` and its argument is an array of parameters consisting of the $H_3^+$ concentration, the forward reaction rates, the backward reaction rates, and the $H_2$ concentration as a percentage of the $HD$ concentration, in that order.

In [ ]:
param0 = np.array([932, 1e-9, 1e-9, 1e-9, 1e-10, 1e-10, 1e-10, 0.01])
result = opt.least_squares(total_fit_2,param0,bounds=(0.01*param0, 100*param0),x_scale=param0,verbose=1)
print(result.x)

You can now make a plot that shows the new model with optimized parameters gives better agreement with experiment compared to before.

In [ ]:
# The code below is an INCOMPLETE solution to Assignment 4. 
# You may use it as a guide to completing the assignment but keep in mind
# that it does not give the right answer.

def calc_residuals(params):
    myNetwork = ReactionNetwork()
    myNetwork.add_species("H3+")
    myNetwork.add_species("H2D+")
    myNetwork.add_species("D2H+")
    myNetwork.add_species("D3+")
    myNetwork.add_species("HD")
    myNetwork.add_species("H2")
    myNetwork.add_reaction("H3+ + HD -> H2D+ + H2", kfwd=params[1], kbak=params[4])
    myNetwork.add_reaction("H2D+ + HD -> D2H+ + H2", kfwd=params[2], kbak=params[5])
    myNetwork.add_reaction("D2H+ + HD -> D3+ + H2", kfwd=params[3], kbak=params[6])
    result = scipy.integrate.solve_ivp(myNetwork.calc_derivative, (0, 0.15), np.array([932, 0, 0, 0, 6.3e10*(1-params[7]), 6.3e10*params[7]]), t_eval=df['time'])
    residual_h3 = result.y[0] - df['H3+']
    residual_h3 = np.array(residual_h3[~np.isnan(residual_h3)])
    residual_h2d = result.y[1] - df['H2D+']
    residual_h2d = np.array(residual_h2d[~np.isnan(residual_h2d)])
    all_residuals = np.hstack((residual_h3, residual_h2d))
    return all_residuals

param0 = np.array([932, 1e-9, 1e-9, 1e-9, 1e-10, 1e-10, 1e-10, 0.01])

calc_residuals(param0)

result = opt.least_squares(calc_residuals,param0,jac='3-point',bounds=(0.01*param0, 100*param0),x_scale=param0,verbose=1)



In [ ]:
result.x